### Imports:

In [265]:
from aima.logic import  PropKB, dpll_satisfiable
from aima.utils import expr,Expr

### Problem Definition:
Three different houses each contain a different pet, a different
drink, and an inhaCitant of a different nationality. The following six
facts are true aCout these houses:
1. The man in the third (middle) house drinks milk.

2. The Spaniard owns the dog.

3. The Ukranian drinks the tea.

4. The Norwegian lives in the first house.

5. The Norwegian lives next to the tea-drinker.

6. The juice-drinker owns the fox.

Questions. Who owns the zebra? Who drinks juice?

### Helping Functions

In [266]:
def symbols_combinations(symbols, nationality):
    """
    The function `symbols_combinations` takes a list of symbols and a list of nationalities, and returns
    a list of combinations where each symbol is concatenated with each nationality.
    
    :param symbols: A list of symbols
    :param nationality: A list of nationalities
    :return: a list of combinations of symbols and nationalities.
    """
    result = []
    for symbol in symbols:
        for human in nationality:
            result.append(symbol + human)
    return result

In [267]:
def x_nationality_form(symbols):
    """
    The function `x_nationality_form` takes a list of symbols and generates a set of rules based on the
    symbols' combinations.
    
    :param symbols: The `symbols` parameter is a list of strings representing nationalities. Each string
    in the list represents a nationality
    :return: a list of rules. Each rule is a string representing a logical expression in the form of a
    disjunction (OR) of conjunctions (AND).
    """
    rules = []
    count = 0
    sentence = ""
    for combination in symbols:
        sentence = sentence + f"({combination}"
        for remaining in symbols:
            if combination != remaining and combination[0] == remaining[0]:
                sentence = sentence + f" & ~{remaining}"
        sentence = sentence + ")|"
        count += 1
        if count % 3 == 0:
            sentence = sentence[:-1]
            rules.append(sentence)
            sentence = ""
    return rules

In [268]:
def get_answer(results, query):
    """
    The function `get_answer` takes in a dictionary of results and a query, and returns the value from
    the results dictionary corresponding to the query.
    
    :param results: The "results" parameter is a dictionary or a data structure that contains the
    possible answers to a query. Each answer is associated with a specific key or expression
    :param query: A string representing the query or question that the user wants to ask
    :return: The function `get_answer` returns the value at the index specified by the `query` in the
    `results` list.
    """
    return results[expr(query)]

### Define combinations of Symbols:

In [269]:
pet_nationality = []
house_nationality = []
drink_nationality = []
symbols = []
# symbols
houses = ['A','B','C']
pets = ['F','Z','D']
drinks = ['M','J','T']
nationality = ['U','S','N']
# combinations of symbols
pet_nationality = symbols_combinations(pets, nationality)
house_nationality = symbols_combinations(houses, nationality)
drink_nationality = symbols_combinations(drinks, nationality)
# add combinations
symbols.extend(pet_nationality)
symbols.extend(house_nationality)
symbols.extend(drink_nationality)
symbols

['FU',
 'FS',
 'FN',
 'ZU',
 'ZS',
 'ZN',
 'DU',
 'DS',
 'DN',
 'AU',
 'AS',
 'AN',
 'BU',
 'BS',
 'BN',
 'CU',
 'CS',
 'CN',
 'MU',
 'MS',
 'MN',
 'JU',
 'JS',
 'JN',
 'TU',
 'TS',
 'TN']

### Form Knowledge base:

In [270]:
rules = []
# General Knowledge base
one_pet_for_person = ["(FN & ~ZN & ~DN) | (~FN & ZN & ~DN) | (~FN & ~ZN & DN)", "(FS & ~ZS & ~DS) | (~FS & ZS & ~DS) | (~FS & ~ZS & DS)","(FU & ~ZU & ~DU) | (~FU & ZU & ~DU) | (~FU & ~ZU & DU)"]
rules.extend(x_nationality_form(pet_nationality))
rules.extend(x_nationality_form(house_nationality))
rules.extend(x_nationality_form(drink_nationality))
rules.extend(one_pet_for_person)
# Specific Knowledge base
s1 = "(CU & MU & ~MN & ~MS & ~AU & ~BU) | (CS & MS & ~MN & ~MU & ~AS & ~BS) | (CN & MN & ~MU & ~MS & ~AN & ~BN)"
s6 = "(JU & FU & ~FN & ~FS & ~TU & ~MU) | (JN & FN & ~FU & ~FS & ~TN & ~MN) | (JS & FS & ~FN & ~FU & ~TS & ~MS)"
rules.append(s1)
rules.extend(["DS", "TU", "AN", "(~TN & TU & AN) ==> BU"])
rules.append(s6)
rules

['(FU & ~FS & ~FN)|(FS & ~FU & ~FN)|(FN & ~FU & ~FS)',
 '(ZU & ~ZS & ~ZN)|(ZS & ~ZU & ~ZN)|(ZN & ~ZU & ~ZS)',
 '(DU & ~DS & ~DN)|(DS & ~DU & ~DN)|(DN & ~DU & ~DS)',
 '(AU & ~AS & ~AN)|(AS & ~AU & ~AN)|(AN & ~AU & ~AS)',
 '(BU & ~BS & ~BN)|(BS & ~BU & ~BN)|(BN & ~BU & ~BS)',
 '(CU & ~CS & ~CN)|(CS & ~CU & ~CN)|(CN & ~CU & ~CS)',
 '(MU & ~MS & ~MN)|(MS & ~MU & ~MN)|(MN & ~MU & ~MS)',
 '(JU & ~JS & ~JN)|(JS & ~JU & ~JN)|(JN & ~JU & ~JS)',
 '(TU & ~TS & ~TN)|(TS & ~TU & ~TN)|(TN & ~TU & ~TS)',
 '(FN & ~ZN & ~DN) | (~FN & ZN & ~DN) | (~FN & ~ZN & DN)',
 '(FS & ~ZS & ~DS) | (~FS & ZS & ~DS) | (~FS & ~ZS & DS)',
 '(FU & ~ZU & ~DU) | (~FU & ZU & ~DU) | (~FU & ~ZU & DU)',
 '(CU & MU & ~MN & ~MS & ~AU & ~BU) | (CS & MS & ~MN & ~MU & ~AS & ~BS) | (CN & MN & ~MU & ~MS & ~AN & ~BN)',
 'DS',
 'TU',
 'AN',
 '(~TN & TU & AN) ==> BU',
 '(JU & FU & ~FN & ~FS & ~TU & ~MU) | (JN & FN & ~FU & ~FS & ~TN & ~MN) | (JS & FS & ~FN & ~FU & ~TS & ~MS)']

### Use Propositional logic for representation

In [271]:
prop_KB = PropKB()
for rule in rules:
    prop_KB.tell(expr(rule))

### Use DPLL for Inference

In [272]:
results = dpll_satisfiable(Expr("&", *prop_KB.clauses))

### Ask Questions

In [273]:
# who owns zebra
print({"Ukranian":get_answer(results,"ZU"), "Spaniard":get_answer(results,"ZS"), "Norwegian":get_answer(results,"ZN")})

# who drink juice
print({"Ukranian":get_answer(results,"JU"), "Spaniard":get_answer(results,"JS"), "Norwegian":get_answer(results,"JN")})

{'Ukranian': True, 'Spaniard': False, 'Norwegian': False}
{'Ukranian': False, 'Spaniard': False, 'Norwegian': True}
